# What is the impact of agricultural corn field expansion rate on deforestation rate in countries?


By Platypus

## Table of Contents
1. [Building our table](#Getting-the-Data)
    - [ISO3](#ISO3)
    - [Variable X: Cornfield data](#Cornfield-Data)
    - [Land size](#Land-Size)
    - [Variable Y: Forest area](#Forest-Area)
    - [Heterogeneity Variable: Land available for corn expansion other than forests](#Heterogeneity-variable)
    - [Confounders](#Confounders)
        - [Corn Price Data](#Corn-Price-Data)
        - [Delta °T](#Delta-T)  
        - [Average Temperature](#Average-Temperature)
        - [GDP](#GDP)
        - [Rainfall](#Rainfall)
        - [Corruption: The Bayesian Corruption Indicator (bci_bci)](#Corruption)
    

In [166]:
# CSV files
filepath = 'https://raw.githubusercontent.com/ZeliaDec/DataScience/main/Data/'
csv_forest = filepath + "Deforest.xlsx"
csv_cornland = filepath + 'FAOSTAT_data_en_10-2-2024.csv'
csv_landcover = filepath + "FAOSTAT_data_en_11-18-2024.csv"

## Getting the Data

### Installing packages

In [167]:
pip install wbdata

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [168]:
pip install pycountry

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Import libraries

In [169]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import wbdata
import numpy as np
import pycountry
import plotly.express as px

### Functions to map country reference with standardize country names (ISO3)

#### From Country Name to ISO3

In [170]:
# Function to get ISO-3 country code from country name
def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

In [171]:
# Lookup the country by ISO3 code
country = pycountry.countries.get(alpha_3='VAT')

# Display the country name
country.name

'Holy See (Vatican City State)'

!! might be not used !!

In [172]:
country_mapping = {
    'United Republic of Tanzania': 'Tanzania',
    'United States of America': 'United States',
    'Venezuela, RB': 'Venezuela',
    'Bahamas, The': 'Bahamas',
    'Congo, Dem. Rep.':'Congo, The Democratic Republic of the',
    'Congo, Rep.': 'Congo',
    'Turkiye': 'Türkiye',
    "Cote d'Ivoire":"Côte d'Ivoire",
    'St. Lucia':'Saint Lucia',
    "Korea, Dem. People's Rep." : "Korea, Democratic People's Republic of",
    'Korea, Rep.': 'Korea, Republic of',
    'Lao PDR':"Lao People's Democratic Republic",
    'Gambia, The': 'Gambia',
    'Egypt, Arab Rep.': 'Egypt',
    'Iran, Islamic Rep.': 'Iran, Islamic Republic of',
    'St. Kitts and Nevis': 'Saint Kitts and Nevis',
    'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines',
    'Micronesia, Fed. Sts.': 'Micronesia, Federated States of',
    'Virgin Islands (U.S.)':'Virgin Islands, U.S.',
    'Yemen, Rep.':'Yemen',
    'Macao SAR, China': 'Macao',
    'St. Martin (French part)':'Saint Martin (French part)',
    'Curacao':'Curaçao',
    'Hong Kong SAR, China': 'Hong Kong'
    # You can add more mappings if necessary
}

In [173]:
country_not_country = [
    'Africa Eastern and Southern',
    'Africa Western and Central',
    'Central Europe and the Baltics',
    'Upper middle income',
    'East Asia & Pacific (IDA & IBRD countries)',
    'East Asia & Pacific',
    'East Asia & Pacific (excluding high income)',
    'Euro area',
    'Europe & Central Asia',
    'Europe & Central Asia (IDA & IBRD countries)',
    'Europe & Central Asia (excluding high income)',
    'European Union',
    'Fragile and conflict affected situations',
    'Heavily indebted poor countries (HIPC)',
    'Early-demographic dividend',
    'High income',
    'IBRD only',
    'IDA & IBRD total',
    'IDA blend',
    'IDA only',
    'IDA total',
    'Late-demographic dividend',
    'Latin America & Caribbean',
    'Latin America & Caribbean (excluding high income)',
    'Least developed countries: UN classification',
    'Low & middle income',
    'Lower middle income',
    'Middle East & North Africa',
    'Middle East & North Africa (IDA & IBRD countries)',
    'Middle income',
    'Low income',
    'North America',
    'OECD members',
    'Other small states',
    'Pacific island small states',
    'Post-demographic dividend',
    'Small states',
    'South Asia',
    'Sub-Saharan Africa (IDA & IBRD countries)',
    'Sub-Saharan Africa (excluding high income)',
    'Pre-demographic dividend',
    'South Asia (IDA & IBRD)',
    'Sub-Saharan Africa',
    'West Bank and Gaza',
    'Caribbean small states',
    'Channel Islands',
    'Arab World',
    'Middle East & North Africa (excluding high income)',
    'Latin America & the Caribbean (IDA & IBRD countries)',
    'Eastern and Southern Africa',
    'Western and Central Africa',
    'Not classified',
    'World'
]


#### From M49 to ISO3

In [174]:
def m49_to_iso3(m49_code):
    for country in pycountry.countries:
        if hasattr(country, 'numeric') and int(country.numeric) == m49_code:
            return country.alpha_3
    return None

### Cornfield Data

In [175]:
corn_df = pd.read_csv(csv_cornland)
corn_df.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),1961,1961,ha,500000,A,Official figure,NaN
1,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),1962,1962,ha,500000,A,Official figure,NaN
2,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),1963,1963,ha,500000,A,Official figure,NaN
3,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),1964,1964,ha,505000,A,Official figure,NaN
4,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),1965,1965,ha,500000,A,Official figure,NaN


In [176]:
# Filter the data for the year 1992
data_1992 = corn_df[corn_df["Year"] == 1992]

# Find countries with missing values in the "Value" column
countries_no_value_1992 = data_1992[data_1992["Value"].isnull()]["Area"].unique()

# Convert to list
countries_no_value_1992_list = countries_no_value_1992.tolist()

print(countries_no_value_1992_list)

[]


In [177]:
data_1992 = data_1992[["Area", "Value"]]
data_1992.rename(columns={"Area": "country", "Value": "corn1992"}, inplace=True)
data_1992["corn1992"] = data_1992["corn1992"]*0.01
data_1992 = data_1992.dropna()
data_1992.head()

,country,corn1992
31,Afghanistan,2000.00
93,Albania,627.36
155,Algeria,2.90
217,Angola,8470.00
274,Antigua and Barbuda,0.30


In [178]:
#Filtering the data between 2000 and 2021
corn_field_data = corn_df
corn_field_data = corn_field_data[(corn_field_data['Year'] >= 2000) & (corn_field_data['Year'] <= 2021)]
corn_field_data.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
39,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2000,2000,ha,96000,A,Official figure,NaN
40,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2001,2001,ha,80000,A,Official figure,NaN
41,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2002,2002,ha,100000,A,Official figure,NaN
42,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2003,2003,ha,250000,A,Official figure,NaN
43,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2004,2004,ha,250000,A,Official figure,NaN


In [179]:
corn_field_data["Note"].unique()

array([nan, 'Unofficial figure'], dtype=object)

In [180]:
corn_field_data["Flag Description"].unique()

array(['Official figure', 'Estimated value', 'Imputed value',
       'Figure from international organizations',
       'Missing value (data cannot exist, not applicable)'], dtype=object)

In [181]:
corn_field_data = corn_field_data[(corn_field_data["Flag Description"] == "Official figure") & (corn_field_data["Note"] != "Unofficial figure")]

In [182]:
#Unique country amount: Does the number of countires in this data match with deforestation data?
print("Number of countries in the corn field dataset: ",len(corn_field_data["Area"].unique()))

Number of countries in the corn field dataset:  162


In [183]:
corn_field_data.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
39,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2000,2000,ha,96000,A,Official figure,NaN
40,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2001,2001,ha,80000,A,Official figure,NaN
41,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2002,2002,ha,100000,A,Official figure,NaN
42,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2003,2003,ha,250000,A,Official figure,NaN
43,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,112,Maize (corn),2004,2004,ha,250000,A,Official figure,NaN


In [184]:
#filter only the relevant columns
df_corn = corn_field_data
df_corn = df_corn[["Area","Value","Year"]]
df_corn

,Area,Value,Year
39,Afghanistan,96000,2000
40,Afghanistan,80000,2001
41,Afghanistan,100000,2002
42,Afghanistan,250000,2003
43,Afghanistan,250000,2004
...,...,...,...
9543,Zimbabwe,1099945,2017
9544,Zimbabwe,1155075,2018
9545,Zimbabwe,991354,2019
9546,Zimbabwe,1018884,2020


In [185]:
areas_to_exclude = df_corn[df_corn["Value"] == 0]["Area"].unique() #These countries include 0 values
#Exclude these countries completely
df_corn = df_corn[~df_corn['Area'].isin(areas_to_exclude)]
df_corn

,Area,Value,Year
39,Afghanistan,96000,2000
40,Afghanistan,80000,2001
41,Afghanistan,100000,2002
42,Afghanistan,250000,2003
43,Afghanistan,250000,2004
...,...,...,...
9543,Zimbabwe,1099945,2017
9544,Zimbabwe,1155075,2018
9545,Zimbabwe,991354,2019
9546,Zimbabwe,1018884,2020


In [186]:
#Missing values
df_corn.isna().sum()

Area     0
Value    0
Year     0
dtype: int64

In [187]:
value_counts_per_area = df_corn.groupby("Area")["Value"].count()

# Filter to get only the areas where the count is not equal to 22
areas_not_equal_to_22 = value_counts_per_area[value_counts_per_area != 22].index
areas_not_equal_to_22

Index(['Antigua and Barbuda', 'Bahamas', 'Barbados', 'Bhutan',
       'Bosnia and Herzegovina', 'Botswana', 'Burundi', 'Cabo Verde',
       'Cambodia', 'Cameroon', 'Central African Republic', 'Chad', 'Congo',
       'Côte d'Ivoire', 'Democratic People's Republic of Korea',
       'Democratic Republic of the Congo', 'Denmark', 'Eritrea', 'Eswatini',
       'Fiji', 'French Guiana', 'Gambia', 'Ghana', 'Guatemala',
       'Guinea-Bissau', 'Haiti', 'Honduras', 'Indonesia',
       'Iran (Islamic Republic of)', 'Iraq', 'Israel', 'Japan', 'Kuwait',
       'Lebanon', 'Lesotho', 'Libya', 'Lithuania', 'Madagascar', 'Malaysia',
       'Maldives', 'Mauritania', 'Montenegro', 'Mozambique', 'Namibia',
       'New Caledonia', 'Nigeria', 'Oman', 'Papua New Guinea', 'Puerto Rico',
       'Rwanda', 'Saint Vincent and the Grenadines', 'Saudi Arabia', 'Serbia',
       'Serbia and Montenegro', 'Sierra Leone', 'South Sudan', 'Sri Lanka',
       'Sudan', 'Sudan (former)', 'Sweden', 'Tajikistan', 'Thailand',
 

In [188]:
df_corn = df_corn[~df_corn["Area"].isin(areas_not_equal_to_22)]
df_corn.head()

,Area,Value,Year
39,Afghanistan,96000,2000
40,Afghanistan,80000,2001
41,Afghanistan,100000,2002
42,Afghanistan,250000,2003
43,Afghanistan,250000,2004


In [189]:
df_corn['iso3'] = df_corn['Area'].apply(get_iso3)
df_corn[df_corn['iso3'].isnull()]

/var/folders/z1/yry6nb4j25bg_5n5f28yn9zw0000gn/T/ipykernel_41805/3983791102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corn['iso3'] = df_corn['Area'].apply(get_iso3)


,Area,Value,Year,iso3
1057,Bolivia (Plurinational State of),307292,2000,None
1058,Bolivia (Plurinational State of),306118,2001,None
1059,Bolivia (Plurinational State of),310465,2002,None
1060,Bolivia (Plurinational State of),295509,2003,None
1061,Bolivia (Plurinational State of),313849,2004,None
...,...,...,...,...
7175,Republic of Korea,15074,2017,None
7176,Republic of Korea,15472,2018,None
7177,Republic of Korea,14840,2019,None
7178,Republic of Korea,15633,2020,None


In [190]:
# Reverting to previous data before dropping and then manually resolving country mismatches
country_mapping_corn = {
    'Bolivia (Plurinational State of)':'Bolivia, Plurinational State of',
    'China, Taiwan Province of':'Taiwan, Province of China',
    'China': 'China_',
    'China, mainland':'China',
    'Democratic Republic of the Congo':'Congo, The Democratic Republic of the',
    'Iran (Islamic Republic of)':'Iran, Islamic Republic of',
    'Micronesia (Federated States of)': 'Micronesia, Federated States of',
    'Netherlands (Kingdom of the)':'Netherlands',
    'Republic of Korea': 'Korea, Republic of',
    'Venezuela (Bolivarian Republic of)':'Venezuela, Bolivarian Republic of',
    # You can add more mappings if necessary
}

df_corn['Area'] = df_corn['Area'].replace(country_mapping_corn)

/var/folders/z1/yry6nb4j25bg_5n5f28yn9zw0000gn/T/ipykernel_41805/1586715189.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corn['Area'] = df_corn['Area'].replace(country_mapping_corn)


In [191]:
df_corn['iso3'] = df_corn['Area'].apply(get_iso3)
df_corn[df_corn['iso3'].isnull()]

/var/folders/z1/yry6nb4j25bg_5n5f28yn9zw0000gn/T/ipykernel_41805/3983791102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corn['iso3'] = df_corn['Area'].apply(get_iso3)


,Area,Value,Year,iso3
1894,China_,23086228,2000,None
1895,China_,24310506,2001,None
1896,China_,24660837,2002,None
1897,China_,24092820,2003,None
1898,China_,25467145,2004,None
1899,China_,26379450,2005,None
1900,China_,28482649,2006,None
1901,China_,29496901,2007,None
1902,China_,29882708,2008,None
1903,China_,31203367,2009,None


In [192]:
df_corn = df_corn.dropna()
df_corn = df_corn.reset_index()
df_corn.drop('index', axis=1, inplace=True)
df_corn.rename(columns={'Area':'country', 'Value':'Total Size Cornfield in ha'}, inplace=True)
df_corn['Total Size Cornfield in sq km'] = df_corn['Total Size Cornfield in ha']*0.01
df_corn.head()

,country,Total Size Cornfield in ha,Year,iso3,Total Size Cornfield in sq km
0,Afghanistan,96000,2000,AFG,960.0
1,Afghanistan,80000,2001,AFG,800.0
2,Afghanistan,100000,2002,AFG,1000.0
3,Afghanistan,250000,2003,AFG,2500.0
4,Afghanistan,250000,2004,AFG,2500.0


In [193]:
df_corn = pd.merge(df_corn, data_1992, on='country', how='left')
df_corn = df_corn.dropna()
df_corn = df_corn.rename(columns={"country": "Country"})  
df_corn = df_corn[["Country", "iso3", "Year", "Total Size Cornfield in sq km", "corn1992"]]
df_corn.head()

,Country,iso3,Year,Total Size Cornfield in sq km,corn1992
0,Afghanistan,AFG,2000,960.0,2000.0
1,Afghanistan,AFG,2001,800.0,2000.0
2,Afghanistan,AFG,2002,1000.0,2000.0
3,Afghanistan,AFG,2003,2500.0,2000.0
4,Afghanistan,AFG,2004,2500.0,2000.0


In [194]:
last_df = df_corn

In [195]:
len(last_df['Country'].unique())

78

# Forest area 
 
For both the variable Y and the heterogeneity variable, We will use a [database about Global Land Cover](https://www.fao.org/faostat/en/#data/LC) provided by FAO, where we have data for 247 countries, and for 14 classes of land cover :
1. Artificial surfaces (including urban and associated areas)
2. Herbaceous crops
3. Woody crops
4. Multiple or layered crops (Not mapped)
5. Grassland
6. Tree-covered areas
7. Mangroves
8. Shrub-covered areas
9. Shrubs and/or herbaceous vegetation, aquatic or regularly flooded
10. Sparsely natural vegetated areas (Not mapped)
11. Terrestrial barren land
12. Permanent snow and glaciers
13. Inland water bodies
14. Coastal water bodies and intertidal areas

To compute the land available for corn expansion, we will compute the sum of land categories that seem suitable for corn expansion, so potentially convertible lands other than forests :  
- __Herbaceous crops__ : represents existing cropland already used for agricultural purposes. These lands are highly suitable for corn expansion and may involve crop rotation or intensification strategies
- __Grassland__ : Grasslands are often used as pastures but can be converted into cropland. These areas are considered moderately suitable for corn expansion, especially in regions with high land-use pressure
- __Shrub-covered areas__ : Shrublands, while less fertile than grasslands, can still be converted for agricultural use with proper inputs and management. These areas are often targeted in marginal expansions for crops like corn

The reasoning behing is that we might see a stronger relationship between corn expansion and deforestation in countries where there is little land available for corn expansion other than forests, specifically in countries where there is low regulation and high forest covers, because in these regions clearing forests can be economically cheaper, the sale of timber from deforestation provides an additional revenue stream, offsetting the costs of converting forests to farmland. In highly regulated or land-constrained regions, converting existing croplands rather than forests is more likely because there are strong regulations protect forests, infrastructure for existing croplands is already in place, and incentives encourage intensification (e.g., improving yields) over land expansion. We will create a dummy that splits our dataset into two groups:
- Group 0 :  Countries with land suitable and available for corn expansion lower than its median
- Group 1 : Countries with land suitable and available for corn expansion higher than its median

In [196]:
landcover_data = pd.read_csv(csv_landcover)
landcover_data.columns

Index(['Domain Code', 'Domain', 'Area Code (M49)', 'Area', 'Element Code',
       'Element', 'Item Code', 'Item', 'Year Code', 'Year', 'Unit', 'Value',
       'Flag', 'Flag Description'],
      dtype='object')

In [197]:
len(landcover_data['Area'].unique())

247

In [198]:
landcover_data['iso3']=landcover_data['Area Code (M49)'].apply(m49_to_iso3)
print("Number of unique countries:", len(landcover_data['iso3'].unique()))
print("Countries that have no ISO3 code:", landcover_data[landcover_data['iso3'].isnull()]['Area'].unique())

Number of unique countries: 237
Countries that have no ISO3 code: ['Belgium-Luxembourg' 'Channel Islands' 'China' 'Czechoslovakia'
 'Ethiopia PDR' 'Johnston Island' 'Midway Island'
 'Netherlands Antilles (former)' 'Serbia and Montenegro' 'Sudan (former)'
 'Wake Island']


In [199]:
# Convert 'Value' from 1000 hectares to square km
landcover_data['Value_sq_km'] = landcover_data['Value'] * 10

In [200]:
forests = landcover_data[landcover_data['Item']=='Tree-covered areas']
forests.rename(columns={'Area': 'Country'}, inplace=True)
forests.rename(columns={'Value_sq_km': 'Tree-covered areas in sq km'}, inplace=True)
forests = forests[['Country','iso3', 'Year', 'Element','Tree-covered areas in sq km']]
forests = forests.dropna()
len(forests['Country'].unique())

/var/folders/z1/yry6nb4j25bg_5n5f28yn9zw0000gn/T/ipykernel_41805/4269415020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forests.rename(columns={'Area': 'Country'}, inplace=True)
/var/folders/z1/yry6nb4j25bg_5n5f28yn9zw0000gn/T/ipykernel_41805/4269415020.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forests.rename(columns={'Value_sq_km': 'Tree-covered areas in sq km'}, inplace=True)


236

In [201]:
forests=forests[forests['Element']=='Area from CCI_LC']
forests=forests[['Country','iso3','Year','Tree-covered areas in sq km']]

# **Heterogeneity variable**
## Land available for corn expansion other than forests

### Why ?
- __Direct Relevance to Corn Expansion__:
    This variable captures the potential for agricultural growth, specifically for corn, by estimating land areas that could feasibly be converted to cornfields.
    Unlike general measures of land availability, it focuses on lands that are ecologically and practically suitable for corn cultivation.

- __Variation Across Countries__:
    The variable inherently reflects differences between countries, such as urbanization levels, existing cropland distribution, and natural geographic constraints, making it an ideal heterogeneity factor.
    Countries with more available land for corn expansion may exhibit stronger links between corn expansion and deforestation, while those with limited availability may rely on intensification rather than land clearing.

- __Focus on Agricultural Pressure__:
    This variable aligns directly with the agricultural pressures driving deforestation, providing a more targeted perspective than broader variables like general cropland area.
### How is it constructed ?
To compute the land available for corn expansion, we will compute the sum of land categories that seem suitable for corn expansion, so potentially convertible lands other than forests :  
- __Herbaceous crops__ : represents existing cropland already used for agricultural purposes. These lands are highly suitable for corn expansion and may involve crop rotation or intensification strategies
- __Grassland__ : Grasslands are often used as pastures but can be converted into cropland. These areas are considered moderately suitable for corn expansion, especially in regions with high land-use pressure
- __Shrub-covered areas__ : Shrublands, while less fertile than grasslands, can still be converted for agricultural use with proper inputs and management. These areas are often targeted in marginal expansions for crops like corn

The reasoning behing is that we might see a stronger relationship between corn expansion and deforestation in countries where there is little land available for corn expansion other than forests, specifically in countries where there is low regulation and high forest covers, because in these regions clearing forests can be economically cheaper, the sale of timber from deforestation provides an additional revenue stream, offsetting the costs of converting forests to farmland. In highly regulated or land-constrained regions, converting existing croplands rather than forests is more likely because there are strong regulations protect forests, infrastructure for existing croplands is already in place, and incentives encourage intensification (e.g., improving yields) over land expansion. We will create a dummy that splits our dataset into two groups:
- Group 0 :  Countries with land suitable and available for corn expansion lower than its median
- Group 1 : Countries with land suitable and available for corn expansion higher than its median

In [202]:
landcover_data['Item'].unique()

array(['Artificial surfaces (including urban and associated areas)',
       'Herbaceous crops', 'Woody crops', 'Multiple or layered crops',
       'Grassland', 'Tree-covered areas', 'Mangroves',
       'Shrub-covered areas',
       'Shrubs and/or herbaceous vegetation, aquatic or regularly flooded',
       'Sparsely natural vegetated areas', 'Terrestrial barren land',
       'Permanent snow and glaciers', 'Inland water bodies',
       'Coastal water bodies and intertidal areas'], dtype=object)

In [203]:
suitable_categories = ['Herbaceous crops', 'Grassland', 'Shrub-covered areas']
suitable_land = landcover_data[landcover_data['Item'].isin(suitable_categories)]

In [204]:
# Pivot the data to have categories as columns
pivoted_suitable_land = suitable_land.pivot_table(index=['Area', 'Year', 'iso3'], columns='Item', values='Value', aggfunc='sum').reset_index()

# Fill NaN values with 0
pivoted_suitable_land = pivoted_suitable_land.fillna(0)

# Sum the suitable categories for each country and year
pivoted_suitable_land['Total Suitable Land'] = pivoted_suitable_land['Herbaceous crops'] + pivoted_suitable_land['Grassland'] + pivoted_suitable_land['Shrub-covered areas']

# Display the updated DataFrame
pivoted_suitable_land

Item,Area,Year,iso3,Grassland,Herbaceous crops,Shrub-covered areas,Total Suitable Land
0,Afghanistan,1992,AFG,24035.39,5763.35,3350.55,33149.29
1,Afghanistan,1993,AFG,24035.41,5769.19,3350.55,33155.15
2,Afghanistan,1994,AFG,24035.44,5767.49,3350.54,33153.47
3,Afghanistan,1995,AFG,24045.96,5771.52,3312.12,33129.60
4,Afghanistan,1996,AFG,24044.98,5785.10,3306.66,33136.74
...,...,...,...,...,...,...,...
7223,Zimbabwe,2018,ZWE,32134.49,24580.74,23857.72,80572.95
7224,Zimbabwe,2019,ZWE,32632.60,24628.39,23961.49,81222.48
7225,Zimbabwe,2020,ZWE,38457.94,19644.49,34617.49,92719.92
7226,Zimbabwe,2021,ZWE,40439.18,20234.77,30766.60,91440.55


In [205]:
pivoted_suitable_land["Year"].min()

np.int64(1992)

In [206]:
pivoted_suitable_land[pivoted_suitable_land["Year"]==1992]["Area"].nunique()

226

In [207]:
pivoted_suitable_land = pivoted_suitable_land[pivoted_suitable_land["Year"]==1992]
pivoted_suitable_land = pivoted_suitable_land[["Area", "iso3", "Total Suitable Land"]]
pivoted_suitable_land = pivoted_suitable_land.dropna()
pivoted_suitable_land.isnull().sum()

Item
Area                   0
iso3                   0
Total Suitable Land    0
dtype: int64

In [208]:
pivoted_suitable_land.reset_index(drop = True, inplace = True)
pivoted_suitable_land.head()

Item,Area,iso3,Total Suitable Land
0,Afghanistan,AFG,33149.29
1,Albania,ALB,1710.49
2,Algeria,DZA,11258.87
3,American Samoa,ASM,5.14
4,Andorra,AND,11.56


In [209]:
pivoted_suitable_land.rename(columns={'Area': 'Country'}, inplace=True)

In [210]:
end_df = last_df

In [211]:
df_final = pd.merge(last_df, pivoted_suitable_land, on=['iso3'], how='outer')
df_final1 = pd.merge(df_final,forests,on=['iso3', 'Year'], how='outer')
df_final1['Total Size Land suitable for corn expansion (sq km)'] = (df_final1['Total Suitable Land']-df_final1['corn1992'])
df_final1['Total Size Land suitable for corn expansion (sq km)'] = df_final1['Total Size Land suitable for corn expansion (sq km)'].clip(lower=0)
df_final1 = df_final1.drop(columns=['Total Suitable Land'])
df_final1

,Country_x,iso3,Year,Total Size Cornfield in sq km,corn1992,Country_y,Country,Tree-covered areas in sq km,Total Size Land suitable for corn expansion (sq km)
0,NaN,ABW,1992.0,NaN,NaN,NaN,Aruba,1.6,NaN
1,NaN,ABW,1993.0,NaN,NaN,NaN,Aruba,1.6,NaN
2,NaN,ABW,1994.0,NaN,NaN,NaN,Aruba,1.6,NaN
3,NaN,ABW,1995.0,NaN,NaN,NaN,Aruba,1.6,NaN
4,NaN,ABW,1996.0,NaN,NaN,NaN,Aruba,1.6,NaN
...,...,...,...,...,...,...,...,...,...
7371,NaN,ZWE,2019.0,NaN,NaN,NaN,Zimbabwe,116499.1,NaN
7372,NaN,ZWE,2020.0,NaN,NaN,NaN,Zimbabwe,116446.9,NaN
7373,NaN,ZWE,2021.0,NaN,NaN,NaN,Zimbabwe,116686.1,NaN
7374,NaN,ZWE,2022.0,NaN,NaN,NaN,Zimbabwe,116696.6,NaN


In [212]:
df_final1 = df_final1.drop(["Country_y", "Country", "corn1992"], axis=1)
df_final1 = df_final1.rename(columns={"Country_x":"Country"})

In [213]:
len(df_final1['Country'].unique())

79

In [214]:
last_df=df_final1

### Average Temperature

Source: https://ourworldindata.org/grapher/monthly-average-surface-temperatures-by-year

In [215]:
avg_temp_df = pd.read_csv(filepath + "monthly-average-surface-temperatures-by-year.csv")
avg_temp_df = avg_temp_df.drop(["Year"], axis = 1)
avg_temp_df = (avg_temp_df
               .groupby(by = ["Entity","Code"])
               .mean()
               .reset_index())
avg_temp_df = pd.melt(
    avg_temp_df,
    id_vars=["Entity", "Code"],  # Keep these columns fixed
    var_name="Reported_Year",    # Avoid using "Year" to prevent conflicts
    value_name="Temperature"     # Rename the values column
)

# Convert Reported_Year to integer
avg_temp_df["Reported_Year"] = avg_temp_df["Reported_Year"].astype(int)

# Apply the query with numeric comparison
avg_temp_df = (avg_temp_df
               .query("2000 <= Reported_Year <= 2021")
               .rename(columns = {"Temperature":"Average_Temperature",
                                  "Code":"iso3",
                                  "Reported_Year" : "Year"}))
avg_temp_df.head()

,Entity,iso3,Year,Average_Temperature
585,Afghanistan,AFG,2021,13.982914
586,Albania,ALB,2021,13.125356
587,Algeria,DZA,2021,25.220117
588,American Samoa,ASM,2021,26.756304
589,Andorra,AND,2021,5.152789


In [216]:
last_df = pd.merge(last_df,avg_temp_df,on = ["iso3","Year"],how = "left")
last_df.head()

,Country,iso3,Year,Total Size Cornfield in sq km,Tree-covered areas in sq km,Total Size Land suitable for corn expansion (sq km),Entity,Average_Temperature
0,NaN,ABW,1992.0,NaN,1.6,NaN,NaN,NaN
1,NaN,ABW,1993.0,NaN,1.6,NaN,NaN,NaN
2,NaN,ABW,1994.0,NaN,1.6,NaN,NaN,NaN
3,NaN,ABW,1995.0,NaN,1.6,NaN,NaN,NaN
4,NaN,ABW,1996.0,NaN,1.6,NaN,NaN,NaN


In [217]:
last_df = last_df.drop(["Entity"], axis=1)
last_df

,Country,iso3,Year,Total Size Cornfield in sq km,Tree-covered areas in sq km,Total Size Land suitable for corn expansion (sq km),Average_Temperature
0,NaN,ABW,1992.0,NaN,1.6,NaN,NaN
1,NaN,ABW,1993.0,NaN,1.6,NaN,NaN
2,NaN,ABW,1994.0,NaN,1.6,NaN,NaN
3,NaN,ABW,1995.0,NaN,1.6,NaN,NaN
4,NaN,ABW,1996.0,NaN,1.6,NaN,NaN
...,...,...,...,...,...,...,...
7371,NaN,ZWE,2019.0,NaN,116499.1,NaN,22.434419
7372,NaN,ZWE,2020.0,NaN,116446.9,NaN,21.785587
7373,NaN,ZWE,2021.0,NaN,116686.1,NaN,21.573132
7374,NaN,ZWE,2022.0,NaN,116696.6,NaN,NaN


### GDP
In million USD

In [218]:
gdp_data = pd.read_csv(filepath + "FAOSTAT_GDP.csv")
gdp_data = gdp_data[gdp_data['Year'] >= 2000]
gdp_data = gdp_data[gdp_data['Year'] <= 2021]
gdp_data.drop('Note', axis=1, inplace=True)
gdp_data = gdp_data.dropna()
gdp_data.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
30,MK,Macro Indicators,4,Afghanistan,6110,Value US$,22008,Gross Domestic Product,2000,2000,million USD,3531.869351,X,Figure from international organizations
31,MK,Macro Indicators,4,Afghanistan,6110,Value US$,22008,Gross Domestic Product,2001,2001,million USD,3620.525250,X,Figure from international organizations
32,MK,Macro Indicators,4,Afghanistan,6110,Value US$,22008,Gross Domestic Product,2002,2002,million USD,4285.191376,X,Figure from international organizations
33,MK,Macro Indicators,4,Afghanistan,6110,Value US$,22008,Gross Domestic Product,2003,2003,million USD,4898.791114,X,Figure from international organizations
34,MK,Macro Indicators,4,Afghanistan,6110,Value US$,22008,Gross Domestic Product,2004,2004,million USD,5504.073142,X,Figure from international organizations


In [219]:
value_counts_per_area = gdp_data.groupby("Area")["Value"].count()

area_not_equal_to_22 = value_counts_per_area[value_counts_per_area !=22].index
area_not_equal_to_22

Index(['Curaçao', 'Netherlands Antilles (former)', 'Sint Maarten (Dutch part)',
       'South Sudan', 'Sudan', 'Sudan (former)'],
      dtype='object', name='Area')

In [220]:
gdp_data = gdp_data[~gdp_data["Area"].isin(area_not_equal_to_22)]
gdp_data = gdp_data[['Area', 'Year', 'Value']]
gdp_data

,Area,Year,Value
30,Afghanistan,2000,3531.869351
31,Afghanistan,2001,3620.525250
32,Afghanistan,2002,4285.191376
33,Afghanistan,2003,4898.791114
34,Afghanistan,2004,5504.073142
...,...,...,...
10865,Zimbabwe,2017,22040.654699
10866,Zimbabwe,2018,23645.265549
10867,Zimbabwe,2019,22594.520375
10868,Zimbabwe,2020,21664.745864


In [221]:
gdp_data['iso3'] = gdp_data['Area'].apply(get_iso3)
gdp_data[gdp_data['iso3'].isnull()]["Area"].unique()

array(['Bolivia (Plurinational State of)', 'China, Hong Kong SAR',
       'China, Macao SAR', 'China, mainland',
       'Democratic Republic of the Congo', 'Iran (Islamic Republic of)',
       'Micronesia (Federated States of)', 'Netherlands (Kingdom of the)',
       'Palestine', 'Republic of Korea',
       'Venezuela (Bolivarian Republic of)'], dtype=object)

In [222]:
country_mapping_temp = {
    'Bolivia (Plurinational State of)':'Bolivia, Plurinational State of',
    'China, Taiwan Province of':'Taiwan, Province of China',
    'China': 'China_',
    'China, mainland':'China',
    'Democratic Republic of the Congo':'Congo, The Democratic Republic of the',
    'Iran (Islamic Republic of)':'Iran, Islamic Republic of',
    'Micronesia (Federated States of)': 'Micronesia, Federated States of',
    'Netherlands (Kingdom of the)':'Netherlands',
    'Republic of Korea': 'Korea, Republic of',
    'Venezuela (Bolivarian Republic of)':'Venezuela, Bolivarian Republic of',
    'China, Hong Kong SAR': 'Hong Kong',
    'Palestine': 'Palestine, State of',
    # You can add more mappings if necessary
}

gdp_data['Area'] = gdp_data['Area'].replace(country_mapping_temp)
gdp_data['iso3'] = gdp_data['Area'].apply(get_iso3)
gdp_data[gdp_data['iso3'].isnull()]["Area"].unique()

array(['China_', 'China, Macao SAR'], dtype=object)

In [223]:
gdp_data = gdp_data.dropna()
gdp_data = gdp_data.reset_index()
gdp_data.drop('index', axis=1, inplace=True)
gdp_data.rename(columns={'Area':'Country', 'Value':'GDP'}, inplace=True)
gdp_data

,Country,Year,GDP,iso3
0,Afghanistan,2000,3531.869351,AFG
1,Afghanistan,2001,3620.525250,AFG
2,Afghanistan,2002,4285.191376,AFG
3,Afghanistan,2003,4898.791114,AFG
4,Afghanistan,2004,5504.073142,AFG
...,...,...,...,...
4505,Zimbabwe,2017,22040.654699,ZWE
4506,Zimbabwe,2018,23645.265549,ZWE
4507,Zimbabwe,2019,22594.520375,ZWE
4508,Zimbabwe,2020,21664.745864,ZWE


In [224]:
last_df = pd.merge(last_df,gdp_data, on=['iso3','Year'], how='outer')
last_df = last_df.drop(["Country_y"], axis=1)
last_df

,Country_x,iso3,Year,Total Size Cornfield in sq km,Tree-covered areas in sq km,Total Size Land suitable for corn expansion (sq km),Average_Temperature,GDP
0,NaN,ABW,1992.0,NaN,1.6,NaN,NaN,NaN
1,NaN,ABW,1993.0,NaN,1.6,NaN,NaN,NaN
2,NaN,ABW,1994.0,NaN,1.6,NaN,NaN,NaN
3,NaN,ABW,1995.0,NaN,1.6,NaN,NaN,NaN
4,NaN,ABW,1996.0,NaN,1.6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
7383,NaN,ZWE,2019.0,NaN,116499.1,NaN,22.434419,22594.520375
7384,NaN,ZWE,2020.0,NaN,116446.9,NaN,21.785587,21664.745864
7385,NaN,ZWE,2021.0,NaN,116686.1,NaN,21.573132,24118.150864
7386,NaN,ZWE,2022.0,NaN,116696.6,NaN,NaN,NaN


In [225]:
last_df = last_df.rename(columns={'Country_x':'Country'})
last_df = last_df.dropna()
last_df = last_df.reset_index()
last_df.drop('index', axis=1, inplace=True)
last_df

,Country,iso3,Year,Total Size Cornfield in sq km,Tree-covered areas in sq km,Total Size Land suitable for corn expansion (sq km),Average_Temperature,GDP
0,Afghanistan,AFG,2000.0,960.00,12281.1,31149.29,12.586175,3531.869351
1,Afghanistan,AFG,2001.0,800.00,11975.3,31149.29,13.413867,3620.525250
2,Afghanistan,AFG,2002.0,1000.00,11851.1,31149.29,13.051083,4285.191376
3,Afghanistan,AFG,2003.0,2500.00,11735.3,31149.29,12.485457,4898.791114
4,Afghanistan,AFG,2004.0,2500.00,11667.1,31149.29,13.233360,5504.073142
...,...,...,...,...,...,...,...,...
1711,Zambia,ZMB,2017.0,14339.44,473443.9,16982.08,21.990675,25895.263867
1712,Zambia,ZMB,2018.0,10860.06,473059.8,16982.08,22.302766,26311.984274
1713,Zambia,ZMB,2019.0,8416.93,471995.7,16982.08,22.617600,23310.346721
1714,Zambia,ZMB,2020.0,13335.19,471548.3,16982.08,22.167939,18063.780790


In [226]:
last_df["Country"].nunique()

78

### Cattle

# COLS Renaming

In [227]:
last_df.keys()

Index(['Country', 'iso3', 'Year', 'Total Size Cornfield in sq km',
       'Tree-covered areas in sq km',
       'Total Size Land suitable for corn expansion (sq km)',
       'Average_Temperature', 'GDP'],
      dtype='object')

In [228]:
last_df.rename(columns={'Total Size Cornfield in sq km':'corn',
                        'Tree-covered areas in sq km':'forest',
                        'Total Size Land suitable for corn expansion (sq km)':'suitability',
                        'Year':'year',
                        'Country':'country',
                        'GDP':'gdp'},inplace=True)

In [229]:
last_df.keys()

Index(['country', 'iso3', 'year', 'corn', 'forest', 'suitability',
       'Average_Temperature', 'gdp'],
      dtype='object')

In [230]:
#last_df.to_csv('./Data/Database.csv', index=False) #save the data to a csv file